# Benchmark

We use a list of public datasets to benchmark all the algorithms in StreamAD. Thanks!

1. AIOPS_KPI, [AIOps Challenge public dataset for KPI anomaly detection](https://github.com/NetManAIOps/KPI-Anomaly-Detection)
2. 

In [1]:

import pandas as pd
import numpy as np
from tqdm import tqdm
from time import perf_counter
from streamad.util import StreamGenerator, CustomDS
from streamad.evaluate import NumentaAwareMetircs, PointAwareMetircs, SeriesAwareMetircs
from dataset import prepare_ds, read_ds
from streamad.model import KNNDetector, xStreamDetector, SpotDetector, RandomDetector, RShashDetector, SRDetector, RrcfDetector, HSTreeDetector, ZScoreDetector, LodaDetector, OCSVMDetector, MadDetector, SArimaDetector


We download the dataset, unzip it, and reconstruct its structure with **prepare_ds()** and load the dataset with **read_ds()**

By now, **ds_name** and **file_name** are represented by

```python

DS = {"AIOPS_KPI": ["preliminary_train", "finals_train", "finals_ground_truth"]}
```



In [2]:
path = './streamad-benchmark-dataset'
ds_name = 'AIOPS_KPI'
prepare_ds(ds_name=ds_name,path=path)

Dataset AIOPS_KPI already exists


In [3]:
dfs = read_ds(ds_name=ds_name,ds_file="preliminary_train")

In [4]:
benchmark_items = [
    "Detector",
    "Dataset",
    "Key",
    "Size(#)",
    "Time(s)",
    "Point_Precision",
    "Point_Recall",
    "Point_Fbeta",
    "Series_Precision",
    "Series_Recall",
    "Series_Fbeta",
    "Numenta_Precision",
    "Numenta_Recall",
    "Numenta_Fbeta",
]
benchmark_df = pd.DataFrame(columns=benchmark_items)

In [5]:
models = {
# KNNDetector: dict(),
# xStreamDetector: dict(),
SpotDetector: dict(back_mean_len=10, global_memory=True, window_len=400),
# RandomDetector: dict(),
# ZScoreDetector: dict(window_len=400),
}

In [6]:
threshold = 0.7
for key, (df, label) in dfs.items():

    ds = CustomDS(df, label)
    stream = StreamGenerator(ds.data)

    for model_class, model_param in models.items():
        model = model_class(**model_param)
        scores = []
        start_time = perf_counter()
        for x in tqdm(stream.iter_item(), total=len(ds.data)):
            score = model.fit_score(x)
            if score is not None:
                score = 1 if score >= threshold else score
            
            scores.append(score)

        time = perf_counter() - start_time

        benchmark_values = [model.__class__.__name__, ds_name, key, len(ds.data), time]

        label = ds.label
        for metric in [
            PointAwareMetircs(),
            SeriesAwareMetircs(),
            NumentaAwareMetircs(),
        ]:
            # scores = np.nan_to_num(np.array(scores, dtype=float), nan=0)
            benchmark_values.extend(list(metric.evaluate(label, scores)))

        benchmark_df.loc[len(benchmark_df)] = benchmark_values

    


100%|██████████| 147024/147024 [00:04<00:00, 29502.33it/s]


In [7]:
benchmark_df

,Detector,Dataset,Key,Size(#),Time(s),Point_Precision,Point_Recall,Point_Fbeta,Series_Precision,Series_Recall,Series_Fbeta,Numenta_Precision,Numenta_Recall,Numenta_Fbeta
0,SpotDetector,AIOPS_KPI,02e99bd4f6cfb33f,128562,5.099677,0.600000,0.000569,0.001136,0.600000,0.001501,0.002994,0.600000,0.001581,0.003153
1,SpotDetector,AIOPS_KPI,046ec29ddf80d62e,8784,0.725400,0.666667,0.025000,0.048193,0.666667,0.012500,0.024540,0.666667,0.025000,0.048193
2,SpotDetector,AIOPS_KPI,07927a9a18fa19ae,10960,0.466661,0.500000,0.007692,0.015152,0.500000,0.003704,0.007353,0.500000,0.003704,0.007353
3,SpotDetector,AIOPS_KPI,09513ae3e75778a3,128971,3.958330,0.214286,0.012605,0.023810,0.214286,0.019400,0.035580,0.214286,0.019400,0.035580
4,SpotDetector,AIOPS_KPI,18fbb1d5a5dc099d,129128,4.816728,0.857143,0.001526,0.003047,0.821429,0.018427,0.036046,0.857143,0.025462,0.049455
5,SpotDetector,AIOPS_KPI,1c35dbf57f55f5e4,128853,6.174144,0.714286,0.001044,0.002084,0.700000,0.038845,0.073605,0.714286,0.057810,0.106963
6,SpotDetector,AIOPS_KPI,40e25005ff8992bd,100254,5.969768,1.000000,0.003058,0.006098,1.000000,0.008621,0.017094,1.000000,0.008621,0.017094
7,SpotDetector,AIOPS_KPI,54e8a140f6237526,8248,1.544427,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,SpotDetector,AIOPS_KPI,71595dd7171f4540,147668,5.534046,0.600000,0.002695,0.005367,0.500000,0.010870,0.021277,0.600000,0.010870,0.021352
9,SpotDetector,AIOPS_KPI,769894baefea4e9e,8784,0.663775,0.333333,0.111111,0.166667,0.333333,0.111111,0.166667,0.333333,0.111111,0.166667


In [8]:
benchmark_df.to_csv('./benchamark_results.csv', index=False)

In [ ]:
# We can check the results and plot the figures, not for necessary.
# from streamad.util import plot
# data, label, date, features = ds.data, ds.label, ds.date, ds.features
# idx=0
# gap=50000
# data = data[idx:idx+gap]
# scores=scores[idx:idx+gap]
# date=date[idx:idx+gap]
# label=label[idx:idx+gap]
# plot(data=data,scores=scores,date=date,features=features,label=label)

## Plot the benchmark results into a table

In [10]:
import plotly.graph_objects as go
import pandas as pd
import numpy as np
benchmark_df = pd.read_csv("./benchamark_results.csv")
# fig = go.Figure(
#     data=[
#         go.Table(
#             header=dict(values=list(benchmark_df.columns)),
#             cells=dict(
#                 values=[
#                     benchmark_df[i].round(decimals=5)
#                     if benchmark_df[i].dtype is np.dtype("float") or benchmark_df[i].dtype is np.dtype("int")
#                     else benchmark_df[i]
#                     for i in benchmark_df.columns.tolist()
#                 ],
#                 format=[""]*3 + [".3f"] * (len(benchmark_df.columns)-3),
#                 fill_color='white',
#                 line_color='lightgrey'
#             ),
#         ),
        
#     ],
# )
# fig.update_layout(margin=dict(l=0, r=0, t=0, b=0))
# fig.write_image("benchmark_results.svg")


Write benchmark results into a table file.

In [14]:
benchmark_df = benchmark_df.applymap(lambda x: round(x,3) if isinstance(x,(int,float)) else x)
content = benchmark_df.to_markdown(index=False)

with open('../docs/source/benchmark.md','w') as f:
    f.write('# Benchmark \n' + content)